# Custom Climate Profiles Generation

Many energy system modeling and other planning processes are designed to intake hourly information in order to capture short term fluctuations in conditions that affect energy supply and demand. An *Annualized Hourly Climate Profile* is a dataset that contains hourly weather conditions at a given location for an entire year, and is commonly referred to as an **8760 Climate Profile**. Climate profiles are able to represent future weather conditions due to a changing climate and can thus better inform design and planning processes for a wide range of future needs. Guidance, methodologies, and recommendations for Cal-Adapt Climate Profiles is [available](https://analytics.cal-adapt.org/analytics/methods/) for reference. 

On the Cal-Adapt: Analytics Engine, we currently provide two kinds of Climate Profiles: 
1. **Standard Year 8760**:  One year of hourly data that represents any desired statistical percentile of weather conditions for a location over a 30-year climatological period. A Standard Year builds a climate profile for each climate model on a single variable and can be used to evaluate both median and extreme conditions by selecting appropriate percentiles. Both historical and future Standard Year 8760s are available.

2. **Typical Meteorological Year 8760 (TMY)**: One year of hourly data that represents the median weather conditions for a location over a climatological period. A TMY is built from ten specific weather variables that are weighted in compliance with TMY standards (insert reference to NREL). TMYs statistically assess the median conditions from 30 years of model data and select the most “typical” month for each month during a year. These are then compiled into a single climate profile. The resulting TMY file includes multiple variables and has very specific formatting requirements. Both historical and future TMY (FTMY) 8760s are available.

**Intended Application**: As a user, I want to <span style="color:#FF0000">**generate a custom climate profile**</span> for my area:
1. Standard Year 8760 profile, with my variable and extreme conditions of choice
2. TMY 8760 profile, for the closest weather station to my area. 

**Runtime**: A Standard Year 8760 can take between **5-60 minutes**, depending on selections to generate. A TMY 8760 profile will take approximately **45 minutes** to generate. Modifications to selections may increase the runtime. 

#### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.util.utils import read_csv_file
from climakitae.core.data_interface import get_subsetting_options
import pandas as pd

## Standard Year Profile 

The Standard Year Profile (SY) represents the mean weather conditions for a one-year period. 

Examining a particular month within the Average Meteorological Year can provide hourly information that could inform energy usage analysis. For example, a warm season month can be used to examine cooling demand; while a cold season month can be used for exploring heating demand change under future climate conditions.

The SY for a given 30-year period (either the historical period 1981-2010, or a thirty year window centered on where each GCM simulation reaches the specified global warming level) is calculated by identifying for each hour the hourly value that is closest to the mean hourly value across all years to produce a full annual timeseries of hourly data for a variable of interest that best represents the average conditions.

The resulting heatmap of AMY values for a full annual timeseries is then plotted, with day of year 1 being January 1st and hour of day given in Pacific Standard Time. 

To learn more about the data available on the Analytics Engine, [see our data catalog](https://analytics.cal-adapt.org/data/). 

**Intended Application**: As a user, I want to **<span style="color:#FF0000">analyze the average weather conditions</span>** of a region of interest by:
1. Computing the average weather conditions
2. Visualize average conditions throughout the year in a heatmap

Because this represents average rather than extreme conditions, a Standard Year Profile is not suited for designing systems to meet the worst-case conditions occurring at a location.

**Runtime**: Standard Year Profile generation time ranges from around **5 minutes** for a single grid cell and and to **1 hour** for the entire state of California.  

### Generating the Standard Year Profile

Select the warming level, location and variable for which you want to generate the Standard Year Profile.

1. **Select variable.** The default is "Air Temperature at 2m". Want to know what variables (and their associated units) are available to choose from? Check out our variable list [here](https://github.com/cal-adapt/climakitae/blob/main/climakitae/data/variable_descriptions.csv).

In [ ]:
variable = "Air Temperature at 2m" 
units = "degF"

2. **Select quantile.** The range is 0.0 to 1.0.

In [ ]:
ptile = 0.5 

3. **Select warming level.** The default is [1.2] and base options are [1.5, 1.75, 2.0, 2.25, 2.5]. If you decide to generate SY for a warming level outside of that list, keep in mind that the realistic range of warming levels is 0.5-4.0. 

In [ ]:
warming_level = [1.45]

4. **Select location.** You have a choice of either 

    1. selecting from one of 33 cached weather stations, 
    2. selecting a cached areas, 
    3. or providing a custom point location of interest. 

    Pleae uncomment the code block in the chunk below that corresponds to your choice. We're provided code for viewing the available options for cached weather stations and areas in the second code chunk below.

In [ ]:
# custom point location: provide a latitude and longitude
latitude = 32.7136  
longitude = -117.2031  

# # cached weather station: provide the station name
# station_name = "Sacramento Executive Airport (KSAC)"

# # cached area: provide the area name
# area_name = "Los Angeles County"

In [ ]:
# View options for cached areas and weather station by incommenting one of the code blocks below.

# Cached weather stations:
get_subsetting_options(area_subset="Stations")

# # Cached areas: you can search from options from the following categories
# # - "states"
# # - "CA counties"
# # - "CA Electricity Demand Forecast Zones"
# # - "CA watersheds"
# # - "CA Electric Balancing Authority Areas"
# # - "CA Electric Load Serving Entities (IOU & POU)"

get_subsetting_options(area_subset="CA counties")

5. **Delta option**. Setting "no_delta" to False will generate a profile in relation to a historical baseline of warming level 1.2C. If you do not want this, set it to True.

In [ ]:
no_delta = True

6. Now we define the selection by the choices you've made above and generate the climate profile. SY generation time ranges from around 5 miinutes for a single grid cell and an hour for the entire state of California. For the location, you need to uncomment either 
    1. "latitude" and "longitude"
    2. "stations"
    3. or "cached_area"

    depending on your selection in step 4.

In [ ]:
# define the selection
selection = {
    "variable": variable,
    "resolution": "3 km",
    "q": ptile,
    "warming_level": warming_level,
    "units": units,
    "latitude": latitude,
    "longitude": longitude,
    # "stations": station_name, # uncomment if you are a using a cached weather station
    # "cached_area": area_name, # uncomment if you are a using a cached area
    "no_delta": no_delta,
}

In [ ]:
# generate the climate profile
profile = get_climate_profile(**selection)

7. **Export**. You can now export your result, if you wish.

In [ ]:
fn = export_profile_to_csv(variable, ptile)
profile.to_csv(fn)

### Typical Meteorological Year Profile (TMY)

<br>The [Typical Meteorological Year](https://nsrdb.nrel.gov/data-sets/tmy) is an hourly dataset used for applications in energy and building systems modeling. Because this represents average rather than extreme conditions, an TMY dataset is not suited for designing systems to meet the worst-case conditions occurring at a location. 

The TMY methodology here mirrors that of the Sandia/NREL TMY3 methodology, and uses historic and projected downscaled climate data available through the Cal-Adapt: Analytics Engine catalog. The [TMY3 method](https://www.nrel.gov/docs/fy08osti/43156.pdf) selects a "typical" month based on ten daily variables: max, min, and mean air and dew point temperatures, max and mean wind speed, global irradiance and direct irradiance. As this methodology heavily weights the solar radiation input data, be aware that the final selection of "typical" months may not be typical for other variables. 

**Intended Application** As a user, I want to <span style="color:#FF0000">**generate a typical meteorological year file**</span> for a location of interest:
- Visualize the TMY dataset across all input variables
- Export the TMY dataset for available models for input into my workflow

**Note**: 
1. For practical generation of a TMY dataset, a user <span style="color:#FF0000">**only needs to provide 2 elements**</span>: the **location**, and **reference time period**. These selections are highlighted below for you. 

**Runtime**: With the default settings, this section takes approximately **50 minutes** to run from start to finish. Modifications to selections may increase the runtime.

### Generating TMY

First, select the location and warming level for which the TMY is generated. TMY considers a set of 10 variables that, unlike for AMY, cannot be changed.

1. **Select Location.** You can either select a station from our pre-generated set of 32 weather stations, are define custom latitude and longitude. Run the block below to list out all station name options.

In [ ]:
# read in station file of CA HadISD stations
stn_file = read_csv_file("data/hadisd_stations.csv")
# Display station names
list(stn_file["station"])

In [ ]:
latitude = 37.9
longitude = -122.06

#stn_name = "Bakersfield Meadows Field (KBFL)"

2. **Select warming level.** The base options are [1.5, 1.75, 2.0, 2.25, 2.5]. If you decide to generate AMY for a warming level outside of that list, keep in mind that the realistic range of warming levels is _. 

In [ ]:
warming_level = 2.0

3. We can use the TMY object to set up, run, and output TMY results to file. The first step is to initialize the object with your selected location and warming level. True is the default value for verbose but we have also set it explicitly for demonstration.

In [ ]:

tmy = TMY(warming_level=warming_level,
          latitude = latitude,
          longitude = longitude,
          #station_name=stn_name, 
          verbose=True)

4. We can run the entire TMY workflow with a single command, as shown below. This will write 4 TMY files, one for each model.
The runtime for this command can reach up to 30 minutes. Because we set verbose to True, the TMY object will print updates as different parts of the workflow initialize.

In [ ]:
tmy.generate_tmy()

5. **(Optional) Export to non-EPW format.** TMY profiles are exported in .epw format by default, but can be exported as both `.csv` and `.tmy` file formats using the method `export_tmy_data` with the argument `extension="csv"` as shown here (uncomment to run):

In [ ]:
#tmy.export_tmy_data(extension="csv")